In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime

#import warnings
#warnings.filterwarnings('ignore')

# 변수지정
code = '225570'
url = f"https://finance.naver.com/item/sise_day.naver?code={code}"
headers = {'User-agent': 'Mozilla/5.0'} # 웹브라우저 접속처럼 인식시킨다.

# get 양식을 완성한다.
req = requests.get(url = url, headers= headers)

# BeautifulSoup 로 추출한 내용을 정리한다.
bs = BeautifulSoup(req.text, 'html.parser') # 마지막 페이지의 url은 pgRR 클래스의 td태그/ 
                                            # 개발자도구로 확인가능

#마지막 페이지 추출하기
pgrr = bs.find('td', class_='pgRR')
print(pgrr)
pgrr.a["href"].split('=')

last_page = int(pgrr.a["href"].split('=')[-1])
last_page 

#일별 시세 추출하기
page_url = '{}&page={}'.format(url, 1)  #1페이지만 추출하기
page_url
data = pd.read_html(requests.get(page_url , headers = {'User-agent': 'Mozilla/5.0'}).text)[0]

<td class="pgRR">
<a href="/item/sise_day.naver?code=225570&amp;page=215">맨뒤
				<img alt="" border="0" height="5" src="https://ssl.pstatic.net/static/n/cmn/bu_pgarRR.gif" width="8"/>
</a>
</td>


C:\Users\brian\AppData\Local\Temp\ipykernel_19964\4046183746.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(requests.get(page_url , headers = {'User-agent': 'Mozilla/5.0'}).text)[0]


In [4]:
#

import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime

# 루프

headers = {'User-agent': 'Mozilla/5.0'}
all_df = pd.DataFrame()

for page in range(1, 215):
    page_url = 'https://finance.naver.com/item/sise_day.naver?code=225570' + '&page=' + str(page)
    head = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(page_url, headers = head)
    df = pd.read_html(requests.get(page_url , headers = {'User-agent': 'Mozilla/5.0'}).text)[0]
    all_df = pd.concat([all_df,df] , ignore_index = True)


# 데이터 전처리

all_df = all_df.rename(columns = {'날짜' : 'date' , '종가':'close', '전일비':'diff'
                              , '시가':'open' , '고가':'high', '저가':'low', '거래량':'volume'})
all_df['date'] = pd.to_datetime(all_df['date'])
all_df = all_df.dropna()
all_df[['close' , 'open' , 'high' , 'low' , 'volume']] = all_df[['close' , 'open' , 'high' , 'low' , 'volume']].astype(int)

all_df = all_df.sort_values(by = 'date')

all_df['date'] = pd.to_datetime(all_df['date'])
all_df['year'] = all_df['date'].dt.year
all_df['month'] = all_df['date'].dt.month
all_df['day_of_week'] = all_df['date'].dt.day_of_week
# 0 = 월 , 1 = 화, 2 = 수 , 3 = 목, 4 = 금 , 5 = 토
    


C:\Users\brian\AppData\Local\Temp\ipykernel_19964\2172882333.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(requests.get(page_url , headers = {'User-agent': 'Mozilla/5.0'}).text)[0]
C:\Users\brian\AppData\Local\Temp\ipykernel_19964\2172882333.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(requests.get(page_url , headers = {'User-agent': 'Mozilla/5.0'}).text)[0]
C:\Users\brian\AppData\Local\Temp\ipykernel_19964\2172882333.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(requests.get(page_url , headers = {'User-agent': 'Mozilla/5.0'}).text)[0]
C:\User

In [9]:
all_df.drop(
    columns = 'date' , inplace = True
)
all_df.reset_index(
    drop = True , inplace= True
)
all_df

,close,diff,open,high,low,volume,year,month,day_of_week
0,2150,상승 10,2130,2150,2115,84668,2015,10,0
1,2115,하락 35,2160,2160,2105,14221,2015,10,1
2,2120,상승 5,2140,2140,2115,3520,2015,10,2
3,2115,하락 5,2120,2120,2110,17423,2015,10,3
4,2125,상승 10,2145,2145,2110,30122,2015,10,4
...,...,...,...,...,...,...,...,...,...
2135,15520,하락 230,15810,15940,15340,453415,2024,6,1
2136,14560,하락 960,15680,15830,14330,1097025,2024,6,2
2137,14260,하락 300,14560,14710,14230,286409,2024,6,3
2138,14150,하락 110,14390,14390,14060,218854,2024,6,4


In [10]:
all_df.to_csv(r'C:\Users\brian\Desktop\JUNSOO\Dataengineering\data\nexon_fi.csv', encoding = "euc-kr")

In [ ]:
# 왜 안되는지 모르겠음
# 10개 페이지 추출하기
page_no = 10
pages = min(last_page, page_no) # page_no 와 비교했을때, 더 작은 값을 선택

df = pd.DataFrame() # 비어있는 데이터 프레임 객체 생성

for page in range(1, pages+1):
    page_url = '{}&page={}'.format(url,page)
    df = pd.concat([pd.read_html(requests.get(page_url , headers = {'User-agent': 'Mozilla/5.0'}).text)[0]] , axis = 1)
    df = df.rename(columns = {'날짜' : 'date' , '종가':'close', '전일비':'diff'
                              , '시가':'open' , '고가':'high', '저가':'low', '거래량':'volume'})
    df['date'] = pd.to_datetime(df['date'])
    df = df.dropna()
    df[['close' , 'open' , 'high' , 'low' , 'volume']] = df[['close' , 'open' , 'high' , 'low' , 'volume']].astype(int)

    df = df.sort_values(by = 'date')